# Topic modelling Q&D pilot
https://medium.datadriveninvestor.com/nlp-with-lda-analyzing-topics-in-the-enron-email-dataset-20326b7ae36f
https://github.com/shoreason/enron-topic-modeling/blob/master/enron_lda.ipynb


In [1]:
import pandas as pd


In [2]:
# Create a small set of emails (10 000)
"""emails_read_all = pd.read_csv('./csv/emails_df.csv')
enron = emails_read_all.head[:10000]
enron.to_csv('./csv/emails_df_10000.csv', index=False)"""

"emails_read_all = pd.read_csv('./csv/emails_df.csv')\nenron = emails_read_all.head[:10000]\nenron.to_csv('./csv/emails_df_10000.csv', index=False)"

In [3]:
enron = pd.read_csv('./csv/emails_df_10000.csv')
#enron = pd.read_csv('./csv/emails_df.csv')

In [4]:
enron.shape #10 000 rows, 18 columns
print(enron.columns)
enron.rename(columns={'content': 'body'}, inplace=True)


Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user'],
      dtype='object')


In [5]:
enron = enron[['From', 'To', 'Subject', 'body']]
print(enron.head())



                                     From  \
0  frozenset({'phillip.allen@enron.com'})   
1  frozenset({'phillip.allen@enron.com'})   
2  frozenset({'phillip.allen@enron.com'})   
3  frozenset({'phillip.allen@enron.com'})   
4  frozenset({'phillip.allen@enron.com'})   

                                       To    Subject  \
0     frozenset({'tim.belden@enron.com'})        NaN   
1  frozenset({'john.lavorato@enron.com'})        Re:   
2   frozenset({'leah.arsdall@enron.com'})   Re: test   
3    frozenset({'randall.gay@enron.com'})        NaN   
4     frozenset({'greg.piper@enron.com'})  Re: Hello   

                                                body  
0                          Here is our forecast\n\n   
1  Traveling to have a business meeting takes the...  
2                     test successful.  way to go!!!  
3  Randy,\n\n Can you send me a schedule of the s...  
4                Let's shoot for Tuesday at 11:45.    


In [6]:
import re
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy # for lemmatization
# for plotting
import pyLDAvis
import pyLDAvis.gensim_models  # the module 'gensim' has renamed to gensim_models
import matplotlib.pyplot as plt

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [7]:
# prep NLTK stop words
from nltk.corpus import stopwords

# You have to install the word on your local machine , 3.84 GB!!
# https://www.nltk.org/data.html
stop_words = stopwords.words('english') # will cause error if not installed accordint to the instructions above
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'send', 'com']) # if replied the mail contains thes words -> remove them



In [8]:
print(enron.iloc[1]['body']) # displays info below

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  

My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.



In [9]:
# Convert email body to list
data = enron.body.values.tolist()
#data = enron.Subject.values.tolist()


In [10]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [11]:
data_words = list(sent_to_words(data))

In [12]:
print(data_words[1])

['traveling', 'to', 'have', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'trip', 'without', 'any', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'trip', 'is', 'even', 'desired', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meetings', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', 'too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', 'the', 'meetings', 'might', 'be', 'better', 'if', 'held', 'in', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go', 'is'

In [13]:
from gensim.models.phrases import Phrases, Phraser

In [14]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [15]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [16]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['beth', 'here', 'are', 'our', 'addresses', 'for', 'dsl', 'lines', 'hunter_shively', 'gawain', 'houston_tx', 'phillip', 'allen', 'merlin_ct_houston_tx', 'mike_grigsby', 'meadow', 'lake', 'houston_tx', 'thanks', 'phillip']


In [17]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
# Remove Stop Words 
data_words_nostops = remove_stopwords(data_words)

In [19]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

#python -m spacy download en

In [20]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#nlp = spacy.load('en', disable=['parser', 'ner']) # this is how it used to be
nlp = spacy.load('en_core_web_sm')


In [21]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[200])

['address', 'dsl', 'line', 'hunter_shively', 'gawain', 'phillip', 'allen', 'mike_grigsby', 'meadow', 'lake', 'thank', 'phillip']


In [22]:
print(data_lemmatized[1])

['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']


In [23]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [24]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [25]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [27]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [28]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.022*"font" + 0.020*"message" + 0.020*"size" + 0.018*"send" + 0.017*"align" + 0.016*"original" + 0.015*"right" + 0.013*"thank" + 0.009*"know" + 0.008*"email"'), (1, '0.014*"market" + 0.010*"iso" + 0.009*"price" + 0.008*"power" + 0.007*"say" + 0.006*"company" + 0.006*"energy" + 0.005*"time" + 0.005*"make" + 0.005*"new"'), (2, '0.142*"ect" + 0.065*"hou" + 0.045*"ee" + 0.019*"forward" + 0.017*"pm" + 0.014*"eric_bass" + 0.013*"caiso" + 0.011*"meeting" + 0.011*"mail" + 0.010*"attach"')]


In [29]:
doc_lda = lda_model[corpus]

In [30]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.872503112892777


In [31]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4292246393403041


In [32]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of impor

In [33]:
pyLDAvis.display(vis)

In [34]:
ldamallet = pyLDAvis.gensim_models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


#ldamallet = pyLDAvis.gensim_models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

AttributeError: module 'pyLDAvis.gensim_models' has no attribute 'wrappers'